<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/twadr_6_mixed_pubmed_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'TwADR-L.fold-6.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [116]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-6.validation.csv",header=None)
valid.head()

,0,1
0,13,It's 6 am and this adderall got me reevaluatin...
1,84,paraplegic
2,91,jitters
3,189,dizziness
4,189,dizzy


In [117]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-6.test.csv",header=None)
test.head()

,0,1
0,13,awake in the middle of the night for an hour or 2
1,24,heart beat for being too strong
2,74,60% of my success is due to taking Adderall
3,85,tremor
4,94,anxiety


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [126]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.240628,4.652459,0.232905,15:42


In [127]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.519144,4.361264,0.259756,15:41


In [128]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.229177,4.325750,0.263021,15:39


In [129]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.185332,4.299704,0.265606,15:38


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [131]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.121370,6.613702,0.026786,00:22


In [132]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.217948,5.122169,0.142857,00:28


In [133]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.722930,4.908741,0.169643,01:11


In [134]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.400141,4.652476,0.187500,01:06
1,5.194238,4.553434,0.232143,01:07


In [135]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.970740,4.290094,0.241071,01:09
1,4.796439,4.178479,0.250000,01:11


In [136]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.585045,4.171834,0.303571,01:10
1,4.415176,4.086590,0.276786,01:12


In [137]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.284203,3.983048,0.330357,01:08
1,4.137220,3.937879,0.321429,01:13


In [138]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.955001,3.913152,0.312500,01:13
1,3.818662,3.835930,0.330357,01:11


In [139]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.711802,3.839417,0.330357,01:08
1,3.648645,3.886913,0.366071,01:09
2,3.457151,3.809455,0.339286,01:13
3,3.318438,3.789612,0.339286,01:12


In [140]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.183474,3.827559,0.339286,01:08
1,3.156451,3.784582,0.357143,01:08
2,3.002085,3.801205,0.366071,01:07
3,2.834119,3.763219,0.366071,01:08


In [141]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.700549,3.791916,0.392857,01:11
1,2.681838,3.876671,0.366071,01:08
2,2.550431,3.798960,0.401786,01:11
3,2.415029,3.867052,0.383929,01:08


In [142]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.321987,3.918347,0.348214,01:08
1,2.302325,3.921114,0.348214,01:08
2,2.209136,3.938128,0.383929,01:11
3,2.049898,3.937029,0.383929,01:12


In [143]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.956877,3.876665,0.401786,01:10
1,1.971086,4.100326,0.303571,01:13
2,1.914010,4.140046,0.348214,01:11
3,1.851558,4.160298,0.321429,01:07


In [144]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.764445,4.180357,0.357143,01:06
1,1.835237,4.306428,0.366071,01:07
2,1.750419,4.223220,0.401786,01:11
3,1.662160,4.221897,0.357143,01:08


In [145]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.578833,4.312213,0.357143,01:10
1,1.625002,4.320027,0.366071,01:08
2,1.576493,4.385085,0.366071,01:12
3,1.534050,4.429982,0.357143,01:13


In [146]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.444017,4.381628,0.366071,01:08
1,1.493441,4.575592,0.392857,01:13
2,1.493362,4.626172,0.366071,01:08
3,1.431224,4.460313,0.392857,01:10


In [147]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.424716,4.577720,0.392857,01:10
1,1.480028,4.609375,0.357143,01:09
2,1.469137,4.669566,0.375000,01:12
3,1.392744,4.666969,0.366071,01:07


In [148]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.386379,4.887236,0.339286,01:08
1,1.414506,4.961771,0.366071,01:07


In [149]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.413859,4.893830,0.357143,01:06
1,1.395357,5.156625,0.330357,01:13


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [152]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1099, 792, 1669, 1204, 119, 1096, 189, 1610, 189, 217, 224, 2103, 1884, 233, 233, 2147, 1711, 355, 1599, 1136, 426, 532, 1676, 600, 600, 706, 1610, 1529, 755, 1813, 792, 809, 1265, 1546, 1610, 1529, 1610, 2017, 908, 1265, 923, 1576, 974, 1610, 978, 119, 978, 119, 233, 1727, 1669, 483, 1099, 1099, 1769, 1509, 1610, 1136, 1099, 1610, 1573, 1373, 1204, 1813, 936, 1265, 1265, 1529, 1295, 1295, 1308, 1308, 371, 1340, 2149, 809, 1610, 1610, 355, 445, 974, 1089, 1597, 1597, 947, 1597, 1610, 1610, 1610, 1610, 1787, 1099, 1787, 2032, 265, 1727, 1727, 1815, 1767, 1727, 1669, 1727, 1089, 1787, 2032, 1787, 1787, 1787, 1787, 119, 56, 824, 1884, 233, 233, 233, 233, 233, 1857, 1610, 1099, 1265, 1718, 452, 2018, 1610, 2032, 2032, 2026, 1508, 1508, 2149, 1610, 2147, 2147, 1546, 2147, 2147, 2147, 2147, 2149, 1787, 2194]
[13, 24, 74, 85, 94, 187, 189, 189, 189, 217, 224, 224, 224, 233, 233, 233, 265, 355, 392, 396, 426, 532, 561, 600, 600, 706, 744, 749, 755, 791, 792, 809, 818, 818, 819, 825, 8

In [153]:
correct_num = 0

for i in range(0, 143):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

56
0.3916083916083916
